In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import glob
import os


In [3]:
(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()

In [4]:
train_images.shape

(60000, 28, 28)

In [5]:
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')

In [6]:
train_images = (train_images - 127.5) / 127.5

In [7]:
batch_size = 256
buffer_size = 60000

In [8]:
datasets = tf.data.Dataset.from_tensor_slices(train_images)

In [9]:
datasets = datasets.shuffle(buffer_size).batch(batch_size)

In [10]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [11]:
# 判别器损失函数
def discriminator_loss(real_image, fake_image):
    real_loss = cross_entropy(tf.ones_like(real_image), real_image)
    fake_loss = cross_entropy(tf.zeros_like(fake_image), fake_image)
    return real_loss + fake_loss

In [12]:
def generator_loss(fake_image):
    return cross_entropy(tf.ones_like(fake_image), fake_image)

In [13]:
generator_opt = tf.keras.optimizers.Adam(1e-4)
discriminator_opt = tf.keras.optimizers.Adam(1e-4)

In [14]:
epochs = 100
noise_dim = 100

num_example_to_generate = 16

seed = tf.random.normal([num_example_to_generate, noise_dim])


In [15]:
def generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, input_shape=(100,), use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Dense(256, input_shape=(100,), use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Dense(28 * 28 * 1, use_bias=False, activation="tanh"))
    model.add(layers.BatchNormalization())
    model.add(layers.Reshape((28, 28, 1)))

    return model
def discriminator_model():
    model = tf.keras.Sequential()

    model.add(layers.Flatten())

    model.add(layers.Dense(512))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Dense(512))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Dense(1))

    return model

generator = generator_model()
discriminator = discriminator_model()

In [16]:
def train_step(images):
    noise = tf.random.normal([batch_size, noise_dim])
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        real_out = discriminator(images, training=True)
        
        gen_image = generator(noise, training=True)
        fake_out = discriminator(gen_image, training=True)

        gen_loss = generator_loss(fake_image=fake_out)
        disc_loss = discriminator_loss(real_image=real_out, fake_image=fake_out)

    gradient_gen = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradient_disc = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_opt.apply_gradients(zip(gradient_gen, generator.trainable_variables))
    discriminator_opt.apply_gradients(zip(gradient_disc, discriminator.trainable_variables))


In [17]:
def generate_plot_image(gen_model, test_noise):
    pre_images = gen_model(test_noise, training=False) # 四维的

    fig = plt.figure(figsize=(4, 4))
    for i in range(pre_images.shape[0]):
        plt.subplot(4, 4, i + 1)    # 从1开始的 不是从0
        plt.imshow((pre_images[i, :, :, 0] + 1) / 2, cmap='gray')
        plt.axis('off')
    plt.show()

In [18]:
def train(dataset, epchos):
    for epoch in range(epchos):
        for image_batch in dataset:
            train_step(image_batch)
            print(".", end='')
        generate_plot_image(gen_model=generator, test_noise=seed)

In [ ]:
train(datasets, epchos=epochs)